In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import time
import matplotlib
import networkx as nx
import matplotlib.pyplot as plt
import random
import math
import numpy as np
from shapely.geometry.polygon import Polygon
from shapely.geometry import LineString
from shapely.geometry import Point
from descartes import PolygonPatch
import matplotlib.animation as animation
import io
import base64
from IPython.display import HTML
from test import *
import yaml
from src.environments import *

## Tutorial for Network Flow Based Muti-agent Path Planning Algorithm

Before we begin, we encourage you to read the tutorial of the network flow based multi-agent path planning algorithm, which is located in ```/lab2-trajectory-planning/docs```. If you want to learn more about the algorithm in detail, with more approximations and theoretical analysis, please take a look at the following paper.

Yu, Jingjin, and Steven M. LaValle. "Multi-agent path planning and network flow." Algorithmic foundations of robotics X. Springer, Berlin, Heidelberg, 2013. 157-173.

## Introduction to Networkx Package

We'll be using the [networkx](https://networkx.github.io) Python package to represent time-expanded graph. We recommend you skim the documentation if you're not familiar with this package.

All you should know about the networkx package for this implementation are 1) adding edge (with some properties), 2) adding node (with some properties) and 3) accessing the node or edge to get value for some property.

First, we can initialize the directed graph as follow (note that time-expanded graph is directed graph) :

In [ ]:
G = nx.DiGraph()

Then you can add edge with property using 'add_edge' function. Let add edge from node "A" to "B", with properties as weight=1 and capacity=1. Note that, non-existing nodes are automatically generated.

In [ ]:
G.add_edge('A', 'B', weight=1, capacity=1)

Similarly, you can add node with property using 'add_node' function. Let add node "C", with properties as pos=(0,0).

In [ ]:
G.add_node('C', pos=(0,0))

Finally, you can get the value of any property of edge or node as follows:

In [ ]:
# To get property of an edge:
Weight_from_A_to_B = G['A']['B']['weight']
print(Weight_from_A_to_B)

# To get property of an node:
Pos_of_C = G.node['C']['pos']
print(Pos_of_C)

Now you are ready! You just need to add edges with proper weight and capacity and solve the problem with the provided solver. Let's start to write the functions for constructing time expanded graph.

## Function for Adding Gadget

We will provide the function to expand one edge of CUG into a gadget as illustrated in the below image. Please have a look how we assign weight (refered to as "cost" in the tutorial) and capacity to each directed edge in the gadget, in accordance with explanation in the tutorial. We have to use "weight" instead of "cost" as the name of edge property, because it is a requirement of a built-in function we will use in the later section. For example, CUG.add_edge(A, B, weight=0, capacity=1).

Also, it is important to follow the naming convention as used in the below image (also for the later section in this material). It will help the test function and visualization tools work correctly.

![title](img/expand_gadget.png)

In [ ]:
def expand_gadget(Time_Expanded_G, CUG, Goal_Nodes, t):
    """ Expand each edge of a CUG into a gadget
    
    This function should iterate over all edges of the CUG, 
    and expand one link of the CUG into a gadget of 5 links which will be stored 
    in the time-expanded graph.
    Each node in the expand gadget must follow the name convention in the above diagram.
    Moreover, there are weight and capacity attached to each link of the gadget.
    
    Input:
        Time_Expanded_G - Time-expanded graph
        CUG - Collision Free Unit Distance Graph which is the originial encoding of the problem
        Goal_Nodes - Set of goal nodes (this parameter is not used in this function)
        t - Time step that the CUG edge will be expaned from t' to t+1
        
    Return value: (none) but the input parameter Time_Expanded_G will be updated
    """

    CUG_Edges = list(CUG.edges.data())

    for e in CUG_Edges:
        u = str(e[0])
        v = str(e[1])
        u_v = str(e[0]) + "_" + str(e[1])
        
        u_t_p = str(u) + "_" + str(t) + "_p"
        v_t_p = str(v) + "_" + str(t) + "_p"
        u_v_t_p = str(u_v) + "_" + str(t) + "_p"
        
        u_tplus1 = str(u) + "_" + str(t+1)
        v_tplus1 = str(v) + "_" + str(t+1)
        u_v_tplus1 = str(u_v) + "_" + str(t+1)        
        
        Time_Expanded_G.add_edge(u_t_p, u_v_t_p, weight=0, capacity=1)
        Time_Expanded_G.add_edge(v_t_p, u_v_t_p, weight=0, capacity=1)
        Time_Expanded_G.add_edge(u_v_t_p, u_v_tplus1, weight=1, capacity=1)
        Time_Expanded_G.add_edge(u_v_tplus1, u_tplus1, weight=0, capacity=1)
        Time_Expanded_G.add_edge(u_v_tplus1, v_tplus1, weight=0, capacity=1)

## Problem 1: Function for Adding Parallel Edges

Please write a function that add parallel edges as shonw in the following figure. From the below example, the added edges are in green and blue color. The green color represents the edges between the t' and t+1 time step while the blue color represent the links between the t+1 and (t+1)' time step. You should assign weight and capacity to each edge. Recall that weigth(cost) for the statianary action from goal node should be 0, because it alreay achieved its goal.

![title](img/expand_stationary.png)

In [ ]:
def expand_parallel(Time_Expanded_G, CUG, Goal_Nodes, t):
    """ Add the links that represent each agent staying at the same node 
    between adjacent time step to the time expanded graph
    
    This function should iterate over all nodes of the CUG and add a link 
    with appropriate weight and capacity.
    Each node in the expand gadget must follow the name convention in the above diagram.
    
    Input:
        Time_Expanded_G - Time-expanded graph
        CUG - Collision Free Unit Distance Graph which is the originial encoding of the problem
        Goal_Nodes - Set of goal nodes
        t - Time step that the CUG edge will be expaned from t' to t+1 and t+1 to (t+1)' 
        
    Return value: (none) but the input parameter Time_Expanded_G will be updated
    """
    ### BEGIN SOLUTION
    CUG_Nodes = list(CUG.nodes())

    for n in CUG_Nodes:
        n_t_p = str(n) + "_" + str(t) + "_p"
        n_tplus1 = str(n) + "_" + str(t+1)
        
        if n in Goal_Nodes:
            Time_Expanded_G.add_edge(n_t_p, n_tplus1, weight=0, capacity=1)
        else:
            Time_Expanded_G.add_edge(n_t_p, n_tplus1, weight=1, capacity=1)
            
    for n in CUG_Nodes:
        n_tplus1 = str(n) + "_" + str(t+1)
        n_tplus1_p = n_tplus1 + "_p"    
        
        Time_Expanded_G.add_edge(n_tplus1, n_tplus1_p, weight=0, capacity=1)  
    ### END SOLUTION

In [ ]:
test_expand_parallel(expand_parallel)
test_ok()

## Function for Setting Initial and Goal Nodes

We will provide the function to specify which nodes are the initial and goal nodes in the time expanded graph by setting its demand attribute.

In [ ]:
def set_terminals(Time_Expanded_G, Initial_Nodes, Goal_Nodes,T):
    """ Set the node attribute "demand" of the initial node and goal node
    
    This function should iterate over all the initial and goal nodes.
    Then set the node attribute "demand" to -1 for the initial node
    and set that of the goal node to 1
    
    Input:
        Time_Expanded_G - Time-expanded graph
        Initial_Nodes - Set of initial nodes
        Goal_Nodes - Set of goal nodes
        T - Maximum number of time steps 
        
    Return value: (none) but the input parameter Time_Expanded_G will be updated
    """
    
    for n in Initial_Nodes:
        n_0_p = str(n) + "_0_p"
        Time_Expanded_G.add_node(n_0_p, demand=-1)
        
    for n in Goal_Nodes:
        n_Tminus1 = str(n) + "_" + str(T)
        Time_Expanded_G.add_node(n_Tminus1, demand=1)
    

## Problem 2: Function for Expanding CUG (Assembling everything)

Now let's put everything all together. Then you will get a function to expand the whole CUG.

In [ ]:
def expand_CUG(CUG, T, Initial_Nodes, Goal_Nodes):
    """ Expand the CUG into a time-expanded graph
    
    This function should iterate over all the time step t from 0 to T-1.
    At each time step, expand the gadget of CUG links and the stationary nodes of CUG
    
    Input:
        CUG - Collision Free Unit Distance Graph which is the originial encoding of the problem
        T - Maximum number of time steps 
        Initial_Nodes - Set of initial nodes    
        Goal_Nodes - Set of goal nodes
        
        
    Return value: Time_Expanded_G - Time-expanded graph which is created and updated inside this function
    """

    ### BEGIN SOLUTION
    Time_Expanded_G = nx.DiGraph()

    for t in range(T):
        expand_gadget(Time_Expanded_G, CUG, Goal_Nodes, t)
        expand_parallel(Time_Expanded_G, CUG, Goal_Nodes, t)

    set_terminals(Time_Expanded_G, Initial_Nodes, Goal_Nodes,T)

    return Time_Expanded_G
    ### END SOLUTION

In [ ]:
test_expand_CUG(expand_CUG)
test_ok()

Now we can use the expanded CUG for solving multi-agent path planning problem. All the remaining helper and simulation functions are provided for you to see how the agents will move without colliding with each other. 

We also provide the make_grid_CUG function for you to create a CUG with specified numbers of rows and columns of the grid.

In [ ]:
def make_grid_CUG(n,m):

    CUG = nx.DiGraph()

    k = 0
    for i in range(m):
        for j in range(n):
            k=k+1
            CUG.add_node(k, pos=(j,i), cat=3)

            if j<n-1 and i<m-1:
                CUG.add_edge(k,k+1)
                CUG.add_edge(k,k+n)
            elif j==n-1 and i<m-1:
                CUG.add_edge(k,k+n)
            elif j<n-1 and i==m-1:
                CUG.add_edge(k,k+1)

    return CUG

Feel free to change the value of n and m to create different layouts of the CUG.

In [ ]:
# make grid CUG
n=3
m=3
CUG = make_grid_CUG(n,m)

#Visualize CUG grid
fig = plt.figure(1, figsize=(30,30), dpi=90)
ax = fig.add_subplot(111)
pos=nx.get_node_attributes(CUG,'pos')
nx.draw(CUG,pos)

The below code will randomly select the initial and goal positions for your agents. After specifying the maximum number of time step T, we can expand the CUG and visualize it. Feel free to change num_agent to any number of agent you want.

In [ ]:
# randomly select the initial and goal nodes
num_agent = 3
Initial_Nodes = []
Goal_Nodes = []
while(len(Initial_Nodes)<num_agent):
    rand_num = np.random.randint(n*m) +1
    if rand_num not in Initial_Nodes:
        Initial_Nodes.append(rand_num)

while(len(Goal_Nodes)<num_agent):
    rand_num = np.random.randint(n*m) +1
    if rand_num not in Goal_Nodes and rand_num not in Initial_Nodes:
        Goal_Nodes.append(rand_num)
              
print("\nInitial Nodes: ")
print(Initial_Nodes)
print("\nGoal Nodes: ")
print(Goal_Nodes)

#Specify the maximum number of time steps
T = m+n

# convert the CUG to the time-expanded graph
Time_Expanded_G = expand_CUG(CUG, T, Initial_Nodes, Goal_Nodes)

The draw_expaned_CUG is provided for visualizing the expanded CUG.

In [ ]:
def draw_expanded_CUG(Time_Expanded_G, CUG, m, n):
    Time_Expanded_G_Draw = Time_Expanded_G
    Expanded_Edges = list(Time_Expanded_G_Draw.edges.data())
    Expanded_Nodes = list(Time_Expanded_G_Draw.nodes.data())
    CUG_Nodes = list(CUG.nodes())

    for nd in Expanded_Nodes:
        node_id = nd[0]           
        node_name_components = node_id.split("_")
        if len(node_name_components) == 2:
            t = int(node_name_components[1])
            pos = CUG.node[int(node_name_components[0])]['pos']
            pos = ( (n-1)*2 + 4*(t-1)*(n-1) + pos[0], pos[1] + 0.5 + (t-1) )
            Time_Expanded_G_Draw.add_node(node_id, pos=pos, cat=1)        

    for nd in Expanded_Nodes:
        node_id = nd[0]           
        node_name_components = node_id.split("_")
        if len(node_name_components)==3:
            t = int(node_name_components[1])
            pos = CUG.node[int(node_name_components[0])]['pos']
            pos = ( 4*t*(n-1) + pos[0], pos[1] + t*1)            
            Time_Expanded_G_Draw.add_node(node_id, pos=pos, cat=2)

    for nd in Expanded_Nodes:
        node_id = nd[0] 
        node_name_components = node_id.split("_")
        if len(node_name_components)==4:
            t = int(node_name_components[2])
                       
            pos11 = Time_Expanded_G_Draw.node[(node_name_components[0] + "_" + str(t) + "_p")]['pos']
            pos12 = Time_Expanded_G_Draw.node[(node_name_components[1] + "_" + str(t) + "_p")]['pos']
            pos21 = Time_Expanded_G_Draw.node[(node_name_components[0] + "_" + str(t+1))]['pos']
            pos22 = Time_Expanded_G_Draw.node[(node_name_components[1] + "_" + str(t+1))]['pos']
            
            pos1 = ( 1/2*(pos11[0] + pos12[0]), 1/2*(pos11[1] + pos12[1]) )
            pos2 = ( 1/2*(pos21[0] + pos22[0]), 1/2*(pos21[1] + pos22[1]) )

            if len(node_name_components)==4:
                pos = ( pos1[0] + 1/3*(pos2[0] - pos1[0]), pos1[1] + 1/3*(pos2[1] - pos1[1]) )

            elif (len(node_name_components)==3 and node_name_components[2] != 'p'):     
                pos = ( pos1[0] + 2/3*(pos2[0] - pos1[0]), pos1[1] + 2/3*(pos2[1] - pos1[1]) )

            Time_Expanded_G_Draw.add_node(node_id, pos=pos, cat=3)
            
        if len(node_name_components)==3 and node_name_components[2] != 'p':
            t = int(node_name_components[2])
                       
            pos11 = Time_Expanded_G_Draw.node[(node_name_components[0] + "_" + str(t-1) + "_p")]['pos']
            pos12 = Time_Expanded_G_Draw.node[(node_name_components[1] + "_" + str(t-1) + "_p")]['pos']
            pos21 = Time_Expanded_G_Draw.node[(node_name_components[0] + "_" + str(t))]['pos']
            pos22 = Time_Expanded_G_Draw.node[(node_name_components[1] + "_" + str(t))]['pos']
            
            pos1 = ( 1/2*(pos11[0] + pos12[0]), 1/2*(pos11[1] + pos12[1]) )
            pos2 = ( 1/2*(pos21[0] + pos22[0]), 1/2*(pos21[1] + pos22[1]) )

            if len(node_name_components)==4:
                pos = ( pos1[0] + 1/3*(pos2[0] - pos1[0]), pos1[1] + 1/3*(pos2[1] - pos1[1]) )

            elif (len(node_name_components)==3 and node_name_components[2] != 'p'):     
                pos = ( pos1[0] + 2/3*(pos2[0] - pos1[0]), pos1[1] + 2/3*(pos2[1] - pos1[1]) )

            Time_Expanded_G_Draw.add_node(node_id, pos=pos, cat=3)        

    return Time_Expanded_G_Draw

Now we can visualize the time expanded CUG.

In [ ]:
#Visualize the time expanded CUG
Time_Expanded_G_Draw = draw_expanded_CUG(Time_Expanded_G, CUG, m, n)
fig2 = plt.figure(2, figsize=(30,30), dpi=90)
ax2 = fig2.add_subplot(111)
pos=nx.get_node_attributes(Time_Expanded_G_Draw,'pos')
color_map = {1:'b', 2:'#FF0099', 3:'#660066'}
nx.draw(Time_Expanded_G_Draw,pos,node_color=[color_map[Time_Expanded_G_Draw.node[node]['cat']] 
                                             for node in Time_Expanded_G_Draw])
plt.show(block=True)

Then use the built-in network_simplex function to find a solution of this network flow problem. One thing to be noted is that the goal position is not fixed for each agent at the beginning. This means that any agent can go to any goal node but they must go to a different one.

In [ ]:
# solving "permutation invariant" multi agent path planning problem using 
# built-in function of networkx package
flowCost, flowDict = nx.network_simplex(Time_Expanded_G)

The provided extract_path function will give you the path of each agent.

In [ ]:
def extract_path(flowDict,Initial_Nodes,CUG,return_type="node"):

    Path = []
    
    for n in Initial_Nodes:
        Path_temp = [n]
        node_name = str(n) + "_0_p";
        Connected_Nodes = flowDict[node_name]     

        while(True):
            flag = 0

            for Node in Connected_Nodes:
                if Connected_Nodes[Node]==1:
                    node_name_components = Node.split("_")
                    if len(node_name_components) == 2:
                        Path_temp.append(int(node_name_components[0]))
                    Connected_Nodes = flowDict[Node]
                    flag = 1
                    break

            if flag == 0:
                break

        Path.append(Path_temp)
        
    
    for n in range(len(Path[0])-1):
        flag = 0
        for p in Path:
            if p[n]!=p[n+1]:
                flag = 1
                break
        if flag==0:
            break

    Path_truncated = []
    for p in Path:
        Path_truncated.append(p[0:n+1])
        
    Path = Path_truncated
            
            
    if return_type=="waypoint":
        
        nodes = CUG.nodes()

        Path_dict = dict()
        for i in range(len(Path)):
            for j in range(len(Path[i])):
                if j==0:
                    Path_dict['agent'+str(i)+'_x'] = [nodes.get(Path[i][j])['pos'][0]]
                    Path_dict['agent'+str(i)+'_y'] = [nodes.get(Path[i][j])['pos'][1]]
                else:
                    Path_dict['agent'+str(i)+'_x'].append(nodes.get(Path[i][j])['pos'][0])
                    Path_dict['agent'+str(i)+'_y'].append(nodes.get(Path[i][j])['pos'][1])
                    
        return Path_dict


    return Path

Then extract the path into a series of nodes at each time step.

In [ ]:
# retrieve path
Path = extract_path(flowDict, Initial_Nodes, CUG, return_type="node")

print ("\nextract path:")
print(Path)

Now let's see how your agents will move by using a simulation function below.

In [ ]:
def simulate(fig,ax, CUG, Path):
    N=20
    nodes = CUG.nodes()

    x_set = []
    y_set = []
    for i in range(len(Path)):
        x_temp = []
        y_temp = []
        for j in range(len(Path[i])-1):
            x1 = nodes.get(Path[i][j])['pos'][0]
            y1 = nodes.get(Path[i][j])['pos'][1]
            x2 = nodes.get(Path[i][j+1])['pos'][0]
            y2 = nodes.get(Path[i][j+1])['pos'][1]

            x_inc = (x2-x1)/N
            y_inc = (y2-y1)/N

            for n in range(N):
                x_temp.append(x1 + x_inc * (n+1))
                y_temp.append(y1 + y_inc * (n+1))

        x_set.append(x_temp)
        y_set.append(y_temp)


    pos=nx.get_node_attributes(CUG,'pos')
    xmin = 10**10
    xmax = -10**10
    ymin = 10**10
    ymax = -10**10
    for p in pos:
        if pos[p][0] < xmin:
            xmin = pos[p][0]
        if pos[p][0] > xmax:
            xmax = pos[p][0]
        if pos[p][1] < ymin:
            ymin = pos[p][1]
        if pos[p][1] > ymax:
            ymax = pos[p][1]

    xrange = [xmin-1, xmax+1]
    yrange = [ymin-1, ymax+1]

    ax.set_xlim(*xrange)
    ax.set_ylim(*yrange)

    scat = []        
    scat = ax.scatter(x_set[0][0],y_set[0][0],color='blue',s=1000)
    
    def animate(i):        
        offsets = []
        for z in range(len(Path)):
            offsets.append([x_set[z][i], y_set[z][i]])
                          
        scat.set_offsets(tuple(offsets))
        return scat

    ani = animation.FuncAnimation(fig, animate, interval=100, frames = len(x_set[0]))
    
    return ani

In [ ]:
# visualize
fig = plt.figure(1, figsize=(10,10), dpi=90)
ax = fig.add_subplot(111)

BLUE = '#6699cc'
for p in Path:
    line = []
    for n in p:
        line.append(CUG.node[int(n)]['pos'])

    linestring = LineString(line)
    dilated = linestring.buffer(0.3)
    patch1 = PolygonPatch(dilated, fc=BLUE, ec=BLUE, alpha=0.5, zorder=2)
    ax.add_patch(patch1)

pos=nx.get_node_attributes(CUG,'pos')
color_map = {1:'b', 2:'#FF0099', 3:'#660066'}
nx.draw(CUG,pos,node_color=[color_map[CUG.node[node]['cat']] for node in CUG])

ani = simulate(fig,ax,CUG,Path)
HTML(ani.to_html5_video())

## (Optional) Application to Real Environment

Now, let's work with the real environment. For this, you should create your own CUG from the given environment file. Then use all of the pre-developed functions to solve the multi-agent path planning problem for the environmnet. Finally, you can extract each agent's waypoint using "extract_path" function by setting "return_type" as "waypoint". Let's do this step by step for the following cells.

To load and utilize the environment yaml file, you will use ```Environment``` class which was provided to you. We encourage you to skim the ```environments.py``` in the ```/lab2-trajetory-planning/src``` folder. The following cell loads the environment file and visualize it.

In [ ]:
# initialize environment class object
env = Environment()

# load example maze environment
env.load_from_yaml_file("src/maze.yaml")

# draw environment
ax = env.draw()

First of all, you have to define the function which makes CUG from environment file. The following function is incomplete function for making CUG from environment yaml file. The missing parts are removing edges and nodes which are not available due to obstacles. (It will be good to see what is the problem and what you should do, by simply using this incomplete function for the further cells.) Please fill the missing parts to make the valid CUG. To do this, you may have to access ```obstacles``` property of the ```environment``` class.

In [ ]:
def make_grid_CUG_with_Env(env,edge_length,margin=(0,0,0,0)):
    """ Make a grid CUG map for the given environment file.
    The edges of the CUG should not traverse any obstacle in the environment. 
    Also, you may want to remove nodes that are not necessary for the path planning.
    
    Input:
        env - environment yaml file
        edge_length - the length of the edge
        margin (optional) - bound margins (xmin_margin, ymin_margin, xmax_margin, ymax_margin)
        
    Return value: CUG, m, n
    """
    

    CUG = nx.DiGraph()
    
    # get bounds of the environment
    bounds = env.bounds
    
    x_min = bounds[0]+margin[0]
    x_max = bounds[2]+margin[2]
    y_min = bounds[1]+margin[1]
    y_max = bounds[3]+margin[3]
    
    k = 0
    
    # find the number of grids
    m = math.floor((x_max - x_min) / edge_length)
    n = math.floor((y_max - y_min) / edge_length)
    
    for i in range(m):
        for j in range(n):
            k=k+1
            CUG.add_node(k, pos=(x_min+i*edge_length,y_min+j*edge_length), cat=3)

            if j<n-1 and i<m-1:
                CUG.add_edge(k,k+1)
                CUG.add_edge(k,k+n)
            elif j==n-1 and i<m-1:
                CUG.add_edge(k,k+n)
            elif j<n-1 and i==m-1:
                CUG.add_edge(k,k+1)
    
    Pruned_CUG = CUG.copy()  # deep copy of the graph
    nodes = CUG.nodes()
    edges = CUG.edges()
                    
    # prune edges which traverse any obstacle
    
    ### BEGIN SOLUTION
    for e in edges:
        path = LineString([nodes.get(e[0])['pos'], nodes.get(e[1])['pos']])
        for polygon in env.obstacles:
            if path.intersects(polygon):
                Pruned_CUG.remove_edge(e[0],e[1])
                break
                
    ### END SOLUTION
                
        
        
    # prune nodes with degree 0
    
    ### BEGIN SOLUTION
    CUG = Pruned_CUG.copy()
    nodes = CUG.nodes()
    edges = CUG.edges()
    for n in nodes:
        if CUG.degree(n)==0:
            Pruned_CUG.remove_node(n)
            
    ### END SOLUTION
    

    return Pruned_CUG, m, n

Then you can make CUG which fits to the environment. It is also useful to define goal and start region, to define the intial and goal nodes. We gave example regions, but feel free to change them to have different initial and goal locations.

In [ ]:
# draw environment
ax = env.draw()

# make grid CUG which fit to the environment
CUG, m, n = make_grid_CUG_with_Env(env,4,margin=(4,4,0,0))

# add goal region
goal_region = Polygon([(-25,-15), (-15,-15), (-15,-5), (-25,-5)])

# add start region
start_region = Polygon([(15,5), (25,5), (25,15), (15,15)])

# Visualize CUG grid
patch1 = PolygonPatch(goal_region, facecolor=[0,0,0.5], edgecolor=[0,0,0], alpha=0.7, zorder=2)
patch2 = PolygonPatch(start_region, facecolor=[0,0,0.5], edgecolor=[0,0,0], alpha=0.7, zorder=2)
ax.add_patch(patch1)
ax.add_patch(patch2)

pos=nx.get_node_attributes(CUG,'pos')
nx.draw(CUG,pos,node_size=30)

Now, you should select initial and goal nodes. If you defined initial and goal regions, you can use them to select initial and goal locations. Otherwise, feel free to select your own initial and goal locations.

In [ ]:
# choose initial and goal nodes
# randomly select the initial and goal nodes
num_agent = 4
Initial_Nodes = []
Goal_Nodes = []

nodes = CUG.nodes()
for n in nodes:
    point = Point(nodes.get(n)['pos'])
    if point.within(start_region):
        if len(Initial_Nodes)<num_agent:
            Initial_Nodes.append(n)
    elif point.within(goal_region):
        if len(Goal_Nodes)<num_agent:
            Goal_Nodes.append(n)

    if len(Initial_Nodes)==num_agent and len(Goal_Nodes)==num_agent:
        break
              
print("\nInitial Nodes: ")
print(Initial_Nodes)
print("\nGoal Nodes: ")
print(Goal_Nodes)

Finally, you can solve the problem and visualize it! (It might take some time for both solving the problem and visualizing the solution. If it takes to much time, try different resolution by changing edge length when you make the CUG.)

In [ ]:
#Specify the maximum number of time steps
T = m+n

# convert the CUG to the time-expanded graph
Time_Expanded_G = expand_CUG(CUG, T, Initial_Nodes, Goal_Nodes)

# solving "permutation invariant" multi agent path planning problem using 
# built-in function of networkx package
flowCost, flowDict = nx.network_simplex(Time_Expanded_G)

# retrieve path
Path = extract_path(flowDict, Initial_Nodes, CUG, return_type="node")

print ("\nextract path:")
print(Path)

In [ ]:
# visualize
fig = plt.figure(1, figsize=(10,10), dpi=90)
ax = fig.add_subplot(111)

BLUE = '#6699cc'
for p in Path:
    line = []
    for n in p:
        line.append(CUG.node[int(n)]['pos'])

    linestring = LineString(line)
    dilated = linestring.buffer(0.3)
    patch1 = PolygonPatch(dilated, fc=BLUE, ec=BLUE, alpha=0.5, zorder=2)
    ax.add_patch(patch1)

pos=nx.get_node_attributes(CUG,'pos')
color_map = {1:'b', 2:'#FF0099', 3:'#660066'}

nx.draw(CUG,pos,node_color=[color_map[CUG.node[node]['cat']] for node in CUG])

ani = simulate(fig,ax,CUG,Path)
HTML(ani.to_html5_video())

Or, you can extract path and create yaml file for waypoints.

In [ ]:
# retrieve path
Path = extract_path(flowDict, Initial_Nodes, CUG, return_type="waypoint")

with open('waypoints.yml','w') as outfile:   
    yaml.dump(Path, outfile, default_flow_style=False)

This is the end of this work! We encourage you to change the functions and use different settings for various scenarios!